In [ ]:
# Going to be training 3 seperate machine learning models and 
# testing their ability to accuratly predict GPP when compared
# with the p-model. 

# models:

### 1. LightGBM
### 2. Random Forest 

# models will be evaluated on two metrics:

### 1. Their ability to predict accuratly for 2014. Are they able
### to replicate the seasonality observed in the FLUXNET GPP?

### 2. Their ability to generalise to a new unseen location: US-Var

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tqdm import tqdm
from scipy import stats
from numpy import mean 

# LightGBM

import lightgbm as lgb
from lightgbm import LGBMRegressor

# P-model
from pyrealm import pmodel

# Random Forest 
from sklearn.ensemble import RandomForestRegressor


# sklearn evaluation metrics
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score, 
                             confusion_matrix, f1_score, ConfusionMatrixDisplay)
from sklearn.model_selection import (train_test_split, GridSearchCV, RandomizedSearchCV, 
                                     cross_val_predict, RepeatedStratifiedKFold, 
                                     KFold, RepeatedKFold, cross_val_score)

# Bayesian Optimization
from skopt import gp_minimize, BayesSearchCV
from skopt.space import Real, Integer, Categorical
from bayes_opt import BayesianOptimization


In [ ]:
# import the data 

# train is the data from 2010-2013

train=pd.read_csv('/Users/abigailbase/PROJECT FILES/FINAL DFs/Modelling dfs/training.csv',index_col=0)

# test is data from 2014 only

test=pd.read_csv('/Users/abigailbase/PROJECT FILES/FINAL DFs/Modelling dfs/test.csv',index_col=0)

# generalisation validation is the US-Var data

us_var=pd.read_csv('/Users/abigailbase/PROJECT FILES/FINAL DFs/Modelling dfs/US_Var_Validation.csv',index_col=0)


In [ ]:
# function for assigning seasons

def assign_season(row):
    hemisphere = row['hemisphere']
    month = row['MONTH']
    
    if hemisphere == 'NH':
        if month in [12, 1, 2]:  # December, January, February
            return 'Winter'
        elif month in [3, 4, 5]:  # March, April, May
            return 'Spring'
        elif month in [6, 7, 8]:  # June, July, August
            return 'Summer'
        elif month in [9, 10, 11]:  # September, October, November
            return 'Autumn'
    elif hemisphere == 'SH':
        if month in [12, 1, 2]:  # December, January, February
            return 'Summer'
        elif month in [3, 4, 5]:  # March, April, May
            return 'Autumn'
        elif month in [6, 7, 8]:  # June, July, August
            return 'Winter'
        elif month in [9, 10, 11]:  # September, October, November
            return 'Spring'
    
    return None  


In [ ]:
# apply function to assign seasons

train['season'] = train.apply(assign_season, axis=1)
test['season'] = test.apply(assign_season, axis=1)
us_var['season'] = us_var.apply(assign_season, axis=1)

In [ ]:
# encode seasons
train=pd.get_dummies(train, columns=['season'],dtype=int)
test=pd.get_dummies(test, columns=['season'],dtype=int)
us_var=pd.get_dummies(us_var, columns=['season'],dtype=int)

In [ ]:
# variables to remove

to_drop=['SITE_ID','date','IGBP','hemisphere','NDVI_point','PPT_log','DAY','WS_F']

In [ ]:
# drop unnessasary cols from the dfs

train=train.drop(columns=to_drop)
test=test.drop(columns=to_drop)
us_var=us_var.drop(columns=to_drop)

In [ ]:
# seperate out the target and predictive vars from test and train

#train

train_x=train.drop(columns=['GPP_DT_VUT_REF'])
train_y=train[['GPP_DT_VUT_REF']]

In [ ]:
#test

test_x=test.drop(columns=['GPP_DT_VUT_REF'])
test_y=test[['GPP_DT_VUT_REF']]

In [ ]:
# limit the period of us-var

us_var = us_var[us_var['YEAR'].isin([2012, 2013, 2014])]


In [ ]:
# seperate us-var into predictive and target vars

us_var_x=us_var.drop(columns=['GPP_DT_VUT_REF'])
us_var_y=us_var[['GPP_DT_VUT_REF']]

In [ ]:
##########################################################################

In [ ]:
#LightGBM

In [ ]:
param_bounds = {
    'num_leaves': (31, 100),
    'learning_rate': (0.01, 0.15),
    'n_estimators': (100, 500),
    'max_depth': (-1, 30),
    'min_child_samples': (50, 150),
    'subsample': (0.6, 1.0),
    'colsample_bytree': (0.6, 1.0),
    'lambda_l1': (0.0, 0.5),
    'lambda_l2': (0.0, 0.5)
}


In [ ]:
### initialise the model

lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', metric='mae', verbosity=-1)


In [ ]:
# optimize LightGBM with Bayesian optimisation 

In [ ]:

def objective_lgb(num_leaves, learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, lambda_l1, lambda_l2):
    
    
    model = lgb.LGBMRegressor(
        boosting_type='gbdt', 
        num_leaves=int(num_leaves),
        learning_rate=float(learning_rate),
        n_estimators=int(n_estimators),
        max_depth=int(max_depth),
        min_child_samples=int(min_child_samples),
        subsample=float(subsample),
        colsample_bytree=float(colsample_bytree),
        reg_alpha=float(lambda_l1),  
        reg_lambda=float(lambda_l2), 
        objective='regression',
        metric='mae',
        verbosity=-1
    )
    
   
    cv_score = -1.0 * cross_val_score(model, train_x, train_y, cv=10, scoring="neg_mean_squared_error").mean()

    return cv_score


In [ ]:
optimizer = BayesianOptimization(
    f=objective_lgb,  
    pbounds=param_bounds, 
    random_state=42
)

optimizer.maximize(init_points=10, n_iter=30)


In [ ]:
# train the model on training data

best_gbm = lgb.LGBMRegressor(
    boosting_type='gbdt',  
    num_leaves=int(best_params['num_leaves']),
    learning_rate=float(best_params['learning_rate']),
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    min_child_samples=int(best_params['min_child_samples']),
    subsample=float(best_params['subsample']),
    colsample_bytree=float(best_params['colsample_bytree']),
    reg_alpha=float(best_params['lambda_l1']),
    reg_lambda=float(best_params['lambda_l2']),
    objective='regression',  
    metric='mae', 
    verbosity=-1  
)


best_gbm.fit(train_x, train_y)


In [ ]:
best_params = optimizer.max['params']
print(f"Best Parameters: {best_params}")

In [ ]:
### 10-fold cross validation on training data

kf = KFold(n_splits=10, shuffle=True, random_state=333)

In [ ]:
train_cv_preds = cross_val_predict(best_gbm, train_x, train_y, cv=kf) #30 seconds


In [ ]:
### evaluate the performance on training data

train_cv_r2 = r2_score(train_y, train_cv_preds)
train_cv_rmse = np.sqrt(mean_squared_error(train_y, train_cv_preds))
train_cv_mae = mean_absolute_error(train_y, train_cv_preds)


print(f"Cross-Validation R² Score on Training Data: {train_cv_r2}") #0.90
print(f"Cross-Validation RMSE on Training Data: {train_cv_rmse}") # 1.24
print(f"Cross-Validation MAE on Training Data: {train_cv_mae}") #0.83



In [ ]:
### evaluate on test data

In [ ]:
LGB_preds = best_gbm.predict(test_x)


In [ ]:
test_r2 = r2_score(test_y, LGB_preds)
test_rmse = np.sqrt(mean_squared_error(test_y, LGB_preds))
test_mae = mean_absolute_error(test_y, LGB_preds)


print(f"Final R² Score on Test Data: {test_r2}") #0.77
print(f"RMSE on Test Data: {test_rmse}") #1.99
print(f"MAE on Test Data: {test_mae}") #1.41



In [ ]:
##############################################################

In [ ]:
# Random Forest 

In [ ]:
# Bayesian optimisation

In [ ]:
def objective(n_estimators, max_depth, min_samples_split, max_features):
    model = RandomForestRegressor(n_estimators=int(n_estimators),
                                  max_depth=int(max_depth),
                                  min_samples_split=int(min_samples_split),
                                  max_features=min(max_features, 1.0),
                                  random_state=2)
    
    return -1.0 * cross_val_score(model, train_x, train_y, cv=10, scoring="neg_mean_squared_error").mean()

In [ ]:
# Bounds for hyperparameters
param_bounds = {
    'n_estimators': (50, 1000),
    'max_depth': (5, 100),
    'min_samples_split': (2, 50),
    'max_features': (0.1, 0.999)
}

In [ ]:
optimizer = BayesianOptimization(
    f=objective, 
    pbounds=param_bounds, 
    random_state=50
)

optimizer.maximize(init_points=10, n_iter=30)

In [ ]:
best_params = optimizer.max['params']
best_params

In [ ]:
best_params_formatted = {
    'n_estimators': int(best_params['n_estimators']),
    'max_depth': int(best_params['max_depth']),
    'min_samples_split': int(best_params['min_samples_split']),
    'max_features': best_params['max_features']
}
best_params_formatted

In [ ]:
final_model = RandomForestRegressor(n_estimators=619,
                                   max_depth=40,
                                   min_samples_split=37,
                                   max_features=37,
                                   random_state=42)

In [ ]:
final_model.fit(train_x, train_y)


In [ ]:
### 10-fold cross validation on training data

kf = KFold(n_splits=10, shuffle=True, random_state=333)

In [ ]:
train_cv_preds = cross_val_predict(final_model, train_x, train_y, cv=kf) #30 seconds


In [ ]:
### evaluate the performance on training data

train_cv_r2 = r2_score(train_y, train_cv_preds)
train_cv_rmse = np.sqrt(mean_squared_error(train_y, train_cv_preds))
train_cv_mae = mean_absolute_error(train_y, train_cv_preds)

print(f"Cross-Validation R² Score on Training Data: {train_cv_r2}") #0.89
print(f"Cross-Validation RMSE on Training Data: {train_cv_rmse}") # 1.29
print(f"Cross-Validation MAE on Training Data: {train_cv_mae}") #0.82

In [ ]:
### evaluate on test data

In [ ]:
RF_preds = final_model.predict(test_x)


In [ ]:
test_r2 = r2_score(test_y, RF_preds)
test_rmse = np.sqrt(mean_squared_error(test_y, RF_preds))
test_mae = mean_absolute_error(test_y, RF_preds)


print(f"Final R² Score on Test Data: {test_r2}") #0.78
print(f"RMSE on Test Data: {test_rmse}") #1.93
print(f"MAE on Test Data: {test_mae}") #1.39



In [ ]:
# RUN THE P-MODEL

In [ ]:
# reverse log of the vpd 

vpd_original = np.exp(test_x['VPD_log'])


In [ ]:
env = pmodel.PModelEnvironment(
    tc=test_x['TA_F'].to_numpy(),  # temp in deg C
    patm=(test_x['PA_F'].to_numpy() * 1000),  # converted to Pa from kPa
    vpd=(vpd_original.to_numpy() * 100),  # converted to Pa from hPa
    co2=test_x['CO2_F_MDS'].to_numpy()  # CO2 concentration (ppm)
)

model = pmodel.PModel(env, method_optchi='prentice14')

model.estimate_productivity(
    fapar=test_x['fapar_y'].to_numpy(),  
    ppfd=test_x['PPFD_IN_y'].to_numpy()  
)

p_preds = model.gpp


In [ ]:
model.summ

In [ ]:
model.optchi.summarize()


In [ ]:
plt.figure(figsize=(8, 6))  

plt.hist(p_preds, bins=30, edgecolor='black', color='skyblue')  # Create the histogram

# Add labels and title
plt.xlabel('Predicted GPP')
plt.ylabel('Frequency')
plt.title('Histogram of Predicted GPP (p_preds)')

# Show the plot
plt.show()

In [ ]:
p_preds= pd.DataFrame(p_preds, columns=['GPP'])  


In [ ]:
# compare results

In [ ]:
# LGB_preds = LGB test prediction
# p_preds = p mod test predcition 
# RF_preds = RF 
# test_y = fluxnet gpp measurements

In [ ]:
test_date=test['date'] #add date col 

In [ ]:
test_igbp=pd.DataFrame(test['IGBP']) 

In [ ]:
test_results = pd.concat([test_date,test_igbp,test_y],axis=1)

In [ ]:
test_results['GBM_preds']=LGB_preds

In [ ]:
test_results['RF_preds']=RF_preds

In [ ]:
test_results['p_preds']=p_preds

In [ ]:
## plots of predicted vs observed results with line 

plt.figure(figsize=(10, 8))
sns.scatterplot(data=test_results, 
                x='GPP_DT_VUT_REF',   
                y='GBM_preds',       
                hue='IGBP',          
                style='IGBP',        
                s=70,                
                palette='Set2')        


slope, intercept = np.polyfit(test_results['GPP_DT_VUT_REF'], test_results['GBM_preds'], 1)
regression_line = slope * test_results['GPP_DT_VUT_REF'] + intercept

plt.plot(test_results['GPP_DT_VUT_REF'], regression_line, color='black', linewidth=1.5, label='_nolegend_')



r2 = r2_score(test_results['GPP_DT_VUT_REF'], test_results['GBM_preds'])
rmse = np.sqrt(mean_squared_error(test_results['GPP_DT_VUT_REF'], test_results['GBM_preds']))
mae = mean_absolute_error(test_results['GPP_DT_VUT_REF'], test_results['GBM_preds'])



plt.xlabel('FLUXNET GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.ylabel('LightGBM Predicted GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.title('LightGBM',fontweight='bold',fontsize=14)

plt.text(19, 1, f'$R^2 = {r2:.2f}$', fontsize=13)
plt.text(19, 2, f'RMSE = {rmse:.2f}', fontsize=13)
plt.text(19, 3, f'MAE = {mae:.2f}', fontsize=13)
plt.legend(fontsize=13,loc='upper left')

plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=test_results, 
                x='GPP_DT_VUT_REF',   
                y='RF_preds',        
                hue='IGBP',          
                style='IGBP',        
                s=70,                
                palette='Set2')      


slope, intercept = np.polyfit(test_results['GPP_DT_VUT_REF'], test_results['RF_preds'], 1)
regression_line = slope * test_results['GPP_DT_VUT_REF'] + intercept

plt.plot(test_results['GPP_DT_VUT_REF'], regression_line, color='black', linewidth=1.5, label='_nolegend_')



r2 = r2_score(test_results['GPP_DT_VUT_REF'], test_results['RF_preds'])
rmse = np.sqrt(mean_squared_error(test_results['GPP_DT_VUT_REF'], test_results['RF_preds']))
mae = mean_absolute_error(test_results['GPP_DT_VUT_REF'], test_results['RF_preds'])



plt.xlabel('FLUXNET GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.ylabel('RF Predicted GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.title('Random Forest',fontweight='bold',fontsize=14)

plt.text(19, 1, f'$R^2 = {r2:.2f}$', fontsize=13)
plt.text(19, 2, f'RMSE = {rmse:.2f}', fontsize=13)
plt.text(19, 3, f'MAE = {mae:.2f}', fontsize=13)
plt.legend(fontsize=13,loc='upper left')


plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=test_results, 
                x='GPP_DT_VUT_REF',  
                y='p_preds',        
                hue='IGBP',         
                style='IGBP',       
                s=70,               
                palette='Set2')     


slope, intercept = np.polyfit(test_results['GPP_DT_VUT_REF'], test_results['p_preds'], 1)
regression_line = slope * test_results['GPP_DT_VUT_REF'] + intercept

plt.plot(test_results['GPP_DT_VUT_REF'], regression_line, color='black', linewidth=1.5, label='_nolegend_')



r2 = r2_score(test_results['GPP_DT_VUT_REF'], test_results['p_preds'])
rmse = np.sqrt(mean_squared_error(test_results['GPP_DT_VUT_REF'], test_results['p_preds']))
mae = mean_absolute_error(test_results['GPP_DT_VUT_REF'], test_results['p_preds'])



plt.xlabel('FLUXNET GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.ylabel('P-model Predicted GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.title('P-model',fontweight='bold',fontsize=14)

plt.text(19, 40, f'$R^2 = {r2:.2f}$', fontsize=13)
plt.text(19, 30, f'RMSE = {rmse:.2f}', fontsize=13)
plt.text(19, 20, f'MAE = {mae:.2f}', fontsize=13)
plt.legend(fontsize=13,loc='upper left')

plt.show()

In [ ]:
test_results['hemisphere']=test['hemisphere']

In [ ]:
# R2 by IGBP


r2_per_igbp_gbm = test_results.groupby('IGBP').apply(
    lambda group: r2_score(group['GPP_DT_VUT_REF'], group['GBM_preds'])
).reset_index().rename(columns={0: 'R²'})

print("R² for each IGBP type (LightGBM):")
print(r2_per_igbp_gbm)



In [ ]:
r2_per_igbp_rf = test_results.groupby('IGBP').apply(
    lambda group: r2_score(group['GPP_DT_VUT_REF'], group['RF_preds'])
).reset_index().rename(columns={0: 'R²'})

print("R² for each IGBP type (RF):")
print(r2_per_igbp_rf)



In [ ]:
# combine LightGBM and Random Forest by IGBP
r2_igbp_combined = pd.DataFrame({
    'IGBP': r2_per_igbp_gbm['IGBP'], 
    'LightGBM': r2_per_igbp_gbm['R²'], 
    'RF': r2_per_igbp_rf['R²']  
})

# reshape the df 
r2_igbp_combined_melted = r2_igbp_combined.melt(id_vars='IGBP', var_name='Model', value_name='R²')

print(r2_igbp_combined_melted)

In [ ]:
# R2 by IGBP

colors={'LightGBM':'red','RF':'blue'}

plt.figure(figsize=(12, 6))
ax=sns.barplot(x='IGBP', y='R²', hue='Model', data=r2_igbp_combined_melted, 
            palette=colors ,alpha=0.8,edgecolor='black')

plt.xlabel('IGBP Type', fontsize=14)
plt.ylabel('R² score', fontsize=14)
plt.xticks(rotation=0)  
plt.legend( fontsize=14)


for p in ax.patches:
    height = p.get_height()  
    if height > 0.01:  
        ax.text(
            p.get_x() + p.get_width() / 2.,  
            height + 0.01,  
            f'{height:.2f}', 
            ha="center", 
            fontsize=11 
        )


plt.tight_layout()
plt.show()


In [ ]:
## seperate results by hemisphere

test_results_NH=test_results[test_results['hemisphere']=='NH']
test_results_SH=test_results[test_results['hemisphere']=='SH']

In [ ]:
# plot the time series (NH)

plt.figure(figsize=(12, 6))

plt.plot(test_results_NH['date'], test_results_NH['GPP_DT_VUT_REF'], 
         label='FLUXNET', 
         color='gray', linewidth=1)

plt.plot(test_results_NH['date'], test_results_NH['GBM_preds'], 
         label='LightGBM ', color='red', linestyle='--')

plt.plot(test_results_NH['date'], test_results_NH['RF_preds'], 
         label='RF', color='blue', linestyle=':',
        alpha=1)

plt.xlabel('Date', fontsize=14)
plt.ylabel('GPP (gC m$^{-2}$ d$^{-1}$)', fontsize=14)

plt.legend(loc='best', fontsize=12)

plt.grid(True, which='both', linestyle='--', linewidth=0.5)


plt.title('',
         fontsize=16,
         fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# plot the time series (SH)

plt.figure(figsize=(12, 6))

plt.plot(test_results_SH['date'], test_results_SH['GPP_DT_VUT_REF'], 
         label='FLUXNET', 
         color='gray', linewidth=1)

plt.plot(test_results_SH['date'], test_results_SH['GBM_preds'], 
         label='LightGBM ', color='red', linestyle='--')

plt.plot(test_results_SH['date'], test_results_SH['RF_preds'], 
         label='RF', color='blue', linestyle=':',
        alpha=1)

plt.xlabel('Date', fontsize=14)
plt.ylabel('GPP (gC m$^{-2}$ d$^{-1}$)', fontsize=14)

plt.legend(loc='best', fontsize=12)

plt.grid(True, which='both', linestyle='--', linewidth=0.5)


plt.title('',
         fontsize=16,
         fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
print("MAX LIGHTGBM PROD:",test_results['GBM_preds'].max())
print("MAX RF PROD:",test_results['RF_preds'].max())
print("MAX OBSERVED PROD:",test_results['GPP_DT_VUT_REF'].max())


In [ ]:
test_results['LAT']=test['LAT']

In [ ]:
test_results['LONG']=test['LONG']

In [ ]:
lat_dist=test_results.copy()

In [ ]:
### latitudinal distribution of predictions

lat_dist['latitude_bin'] = pd.cut(lat_dist['LAT'], bins=np.arange(-90, 91, 10), labels=np.arange(-85, 90, 10))


In [ ]:
bins = [-90, -10, 0, 40, 45, 50, 55, 90]
lat_dist['latitude_bin'] = pd.cut(
    lat_dist['LAT'], 
    bins=bins, 
    labels=[
        "90°S-10°S", 
        "10°S-0°", 
        "0°-40°N", 
        "40°N-45°N", 
        "45°N-50°N", 
        "50°N-55°N", 
        "55°N-90°N"
    ]
)

In [ ]:
latitudinal_distribution_obs = lat_dist.groupby('latitude_bin')['GPP_DT_VUT_REF'].mean()


In [ ]:
latitudinal_distribution_GBM = lat_dist.groupby('latitude_bin')['GBM_preds'].mean()


In [ ]:
latitudinal_distribution_RF = lat_dist.groupby('latitude_bin')['RF_preds'].mean()


In [ ]:
latitudinal_distribution_obs=latitudinal_distribution_obs.reset_index()

In [ ]:
latitudinal_distribution_obs

In [ ]:
latitudinal_distribution_obs=latitudinal_distribution_obs.dropna()

In [ ]:
latitudinal_distribution_GBM=latitudinal_distribution_GBM.reset_index()

In [ ]:
latitudinal_distribution_GBM=latitudinal_distribution_GBM.dropna()

In [ ]:
latitudinal_distribution_RF=latitudinal_distribution_RF.reset_index()

In [ ]:
latitudinal_distribution_RF=latitudinal_distribution_RF.dropna()

In [ ]:
combined_latitudinal_df = latitudinal_distribution_obs.merge(latitudinal_distribution_GBM, on='latitude_bin').merge(latitudinal_distribution_RF, on='latitude_bin')


In [ ]:
combined_latitudinal_df

In [ ]:
colors = ['#cce7ff',  
          '#66c2a4',  
          '#41ae76',  
          '#238b45',  
          '#99d8c9'] 

In [ ]:
plt.figure(figsize=(6, 8))

plt.plot(combined_latitudinal_df['GPP_DT_VUT_REF'], combined_latitudinal_df['latitude_bin'], 
         label='FLUXNET', linestyle='-', color='gray',alpha=0.7)

plt.plot(combined_latitudinal_df['GBM_preds'], combined_latitudinal_df['latitude_bin'], 
         label='LightGBM', marker='s', linestyle='--', color='red')

plt.plot(combined_latitudinal_df['RF_preds'], combined_latitudinal_df['latitude_bin'], 
         label='RF', marker='o', linestyle=':', color='blue')

for i in range(len(combined_latitudinal_df['latitude_bin'])):
    if i < len(combined_latitudinal_df['latitude_bin']) - 1:
        plt.fill_betweenx(
            [combined_latitudinal_df['latitude_bin'].iloc[i], combined_latitudinal_df['latitude_bin'].iloc[i + 1]],  # Between bins
            combined_latitudinal_df[['GPP_DT_VUT_REF', 'GBM_preds', 'RF_preds']].min().min(),  # Starting X value
            combined_latitudinal_df[['GPP_DT_VUT_REF', 'GBM_preds', 'RF_preds']].max().max(),  # Ending X value
            color=colors[i % len(colors)],  
            alpha=0.2  
        )


plt.xlabel('Average GPP (gC m$^{-2}$ d$^{-1}$)')
plt.ylabel('Latitude',fontsize=14)
plt.legend()

In [ ]:
### SHAP feature importance 

In [ ]:
#  SHAP explainer for LightGBM
explainer_best_estimator = shap.Explainer(best_gbm)
shap_values_best_estimator = explainer_best_estimator(test_x) 

In [ ]:
#  SHAP explainer for RF
explainer_final_model = shap.Explainer(final_model)
shap_values_final_model = explainer_final_model(test_x)  


In [ ]:
shap_vals_lightgbm = np.mean(np.abs(shap_values_best_estimator.values), axis=0)
shap_vals_rf = np.mean(np.abs(shap_values_final_model.values), axis=0)


In [ ]:
features = test_x.columns


In [ ]:
shap_df = pd.DataFrame({
    'Features': features,
    'LightGBM': shap_vals_lightgbm,
    'RandomForest': shap_vals_rf
})


In [ ]:
shap_df = shap_df.sort_values(by='LightGBM', ascending=True)


In [ ]:
rename={'PPFD_IN_y':'PPFD_IN',
        'fapar_y':'fAPAR',
        'NIRv_y':'NIRv',
        'Daily_Averaged_SIF_y':'Daily_Averaged_SIF'}

In [ ]:
shap_df['Features'] = shap_df['Features'].replace(rename)


In [ ]:
satellite_features = ['NIRv', 'Daily_Averaged_SIF', 'fAPAR']


In [ ]:
shap_df_lightgbm = shap_df[['Features', 'LightGBM']].copy()
shap_df_rf = shap_df[['Features', 'RandomForest']].copy()


In [ ]:
shap_df_lightgbm = shap_df_lightgbm.sort_values(by='LightGBM', ascending=True)


In [ ]:
shap_df_rf = shap_df_rf.sort_values(by='RandomForest', ascending=True)


In [ ]:
# SHAP plots

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14, 8), sharex=True)

index_lightgbm = np.arange(len(shap_df_lightgbm['Features']))
index_rf = np.arange(len(shap_df_rf['Features']))

ax1.barh(index_lightgbm, shap_df_lightgbm['LightGBM'], color='red', alpha=0.7,
        edgecolor='black')
ax1.set_yticks(index_lightgbm)

y_labels_lightgbm = shap_df_lightgbm['Features']
colors_lightgbm = ['red' if label in satellite_features else 'black' for label in y_labels_lightgbm]

ax1.set_yticklabels(y_labels_lightgbm, fontsize=13)
for tick_label, color in zip(ax1.get_yticklabels(), colors_lightgbm):
    tick_label.set_color(color)

ax1.set_xlabel('Mean |SHAP value|', fontsize=14)
ax1.set_ylabel('Features', fontsize=14)
ax1.set_title('LightGBM',fontweight='bold')

ax2.barh(index_rf, shap_df_rf['RandomForest'], color='blue', alpha=0.7, hatch='//',
        edgecolor='black')
ax2.set_yticks(index_rf)

ax2.yaxis.tick_right()

y_labels_rf = shap_df_rf['Features']
colors_rf = ['red' if label in satellite_features else 'black' for label in y_labels_rf]


ax2.set_yticklabels(y_labels_rf, fontsize=12)
for tick_label, color in zip(ax2.get_yticklabels(), colors_rf):
    tick_label.set_color(color)

ax2.set_xlabel('Mean |SHAP value|', fontsize=13)
ax2.set_ylabel('', fontsize=13)
ax2.set_title('Random Forest',fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# US-ME GENERALISABILITY

In [ ]:
# predict with LightGBM

In [ ]:
us_LGB_preds = best_gbm.predict(us_var_x)

In [ ]:
us_LGB_r2 = r2_score(us_var_y, us_LGB_preds)
us_LGB_rmse = np.sqrt(mean_squared_error(us_var_y, us_LGB_preds))
us_LGB_mae = mean_absolute_error(us_var_y, us_LGB_preds)

print(f"Final R² Score on us_var Data (Random Forest): {us_LGB_r2}") # -1.81
print(f"RMSE on us_var Data (Random Forest): {us_LGB_rmse}") # 4.32
print(f"MAE on us_var Data (Random Forest): {us_LGB_mae}") #  3.72

In [ ]:
# predict with RF

In [ ]:
us_RF_preds = final_model.predict(us_var_x)

In [ ]:
us_RF_r2 = r2_score(us_var_y, us_RF_preds)
us_RF_rmse = np.sqrt(mean_squared_error(us_var_y, us_RF_preds))
us_RF_mae = mean_absolute_error(us_var_y, us_RF_preds)

print(f"Final R² Score on us_var Data (Random Forest): {us_RF_r2}") # -4.317
print(f"RMSE on us_var Data (Random Forest): {us_RF_rmse}") # 5.94
print(f"MAE on us_var Data (Random Forest): {us_RF_mae}") #  5.11

In [ ]:
#p-model prediction 

In [ ]:
vpd_original = np.exp(us_var_x['VPD_log'])


In [ ]:
env = pmodel.PModelEnvironment(
    tc=us_var_x['TA_F'].to_numpy(),  # temp in deg C
    patm=(us_var_x['PA_F'].to_numpy() * 1000),  # converted to Pa from kPa
    vpd=(vpd_original.to_numpy() * 100),  # converted to Pa from hPa
    co2=us_var_x['CO2_F_MDS'].to_numpy()  # CO2 concentration (ppm)
)

model = pmodel.PModel(env, method_optchi='prentice14')

model.estimate_productivity(
    fapar=us_var_x['fapar_y'].to_numpy(),  
    ppfd=us_var_x['PPFD_IN_y'].to_numpy()  
)

us_p_preds = model.gpp

In [ ]:
us_p_r2 = r2_score(us_var_y, us_p_preds)
us_p_rmse = np.sqrt(mean_squared_error(us_var_y, us_p_preds))
us_p_mae = mean_absolute_error(us_var_y, us_p_preds)

print(f"Final R² Score on us_var Data (Random Forest): {us_p_r2}") # -0.06
print(f"RMSE on us_var Data (Random Forest): {us_p_rmse}") # 2.66
print(f"MAE on us_var Data (Random Forest): {us_p_mae}") #  1.56

In [ ]:
# US-Var results DF

In [ ]:
us_results=us_var_y.copy()

In [ ]:
us_results['date']=us_var['date']

In [ ]:
us_results['p_preds']=us_p_preds

In [ ]:
us_results['GBM_preds']=us_LGB_preds

In [ ]:
us_results['RF_preds']=us_RF_preds

In [ ]:
# scatter plots of predicted vs obseerved vals for US-Var

In [ ]:
#LightGBM

plt.figure(figsize=(10, 8))
sns.scatterplot(data=us_results, 
                x='GPP_DT_VUT_REF',   
                y='GBM_preds',        
                s=50,
               color='black')        


# add regression line
slope, intercept = np.polyfit(us_results['GPP_DT_VUT_REF'], us_results['GBM_preds'], 1)
regression_line = slope * us_results['GPP_DT_VUT_REF'] + intercept

# Plot the regression line
plt.plot(us_results['GPP_DT_VUT_REF'], regression_line, color='red', linewidth=1.5, label='_nolegend_')


# add metrics

r2 = r2_score(us_results['GPP_DT_VUT_REF'], us_results['GBM_preds'])
rmse = np.sqrt(mean_squared_error(us_results['GPP_DT_VUT_REF'], us_results['GBM_preds']))
mae = mean_absolute_error(us_results['GPP_DT_VUT_REF'], us_results['GBM_preds'])



# Add labels and title
plt.xlabel('FLUXNET GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.ylabel('Predicted GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.title('LightGBM',fontweight='bold',fontsize=14)

# Add R², RMSE, and MAE as annotations
plt.text(10, 1, f'$R^2 = {r2:.2f}$', fontsize=13)
plt.text(10, 2, f'RMSE = {rmse:.2f}', fontsize=13)
plt.text(10, 3, f'MAE = {mae:.2f}', fontsize=13)

# Show the plot
plt.show()

In [ ]:
# RF

plt.figure(figsize=(10, 8))
sns.scatterplot(data=us_results, 
                x='GPP_DT_VUT_REF',  
                y='RF_preds',        
                s=50,
               color='black')       


# add regression line
slope, intercept = np.polyfit(us_results['GPP_DT_VUT_REF'], us_results['RF_preds'], 1)
regression_line = slope * us_results['GPP_DT_VUT_REF'] + intercept

# Plot the regression line
plt.plot(us_results['GPP_DT_VUT_REF'], regression_line, color='red', linewidth=1.5, label='_nolegend_')


# add metrics

r2 = r2_score(us_results['GPP_DT_VUT_REF'], us_results['RF_preds'])
rmse = np.sqrt(mean_squared_error(us_results['GPP_DT_VUT_REF'], us_results['RF_preds']))
mae = mean_absolute_error(us_results['GPP_DT_VUT_REF'], us_results['RF_preds'])



# Add labels and title
plt.xlabel('FLUXNET GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.ylabel('Predicted GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.title('RF',fontweight='bold',fontsize=14)

# Add R², RMSE, and MAE as annotations
plt.text(10, 1, f'$R^2 = {r2:.2f}$', fontsize=13)
plt.text(10, 2, f'RMSE = {rmse:.2f}', fontsize=13)
plt.text(10, 3, f'MAE = {mae:.2f}', fontsize=13)

# Show the plot
plt.show()

In [ ]:
# P-model

plt.figure(figsize=(10, 8))
sns.scatterplot(data=us_results, 
                x='GPP_DT_VUT_REF',  
                y='p_preds',        
                s=50,
               color='black')        


# add regression line
slope, intercept = np.polyfit(us_results['GPP_DT_VUT_REF'], us_results['p_preds'], 1)
regression_line = slope * us_results['GPP_DT_VUT_REF'] + intercept

# Plot the regression line
plt.plot(us_results['GPP_DT_VUT_REF'], regression_line, color='red', linewidth=1.5, label='_nolegend_')


# add metrics

r2 = r2_score(us_results['GPP_DT_VUT_REF'], us_results['p_preds'])
rmse = np.sqrt(mean_squared_error(us_results['GPP_DT_VUT_REF'], us_results['p_preds']))
mae = mean_absolute_error(us_results['GPP_DT_VUT_REF'], us_results['p_preds'])



# Add labels and title
plt.xlabel('FLUXNET GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.ylabel('Predicted GPP (gC m$^{-2}$ d$^{-1}$)',fontsize=14)
plt.title('P-model',fontweight='bold',fontsize=14)

# Add R², RMSE, and MAE as annotations
plt.text(10, 9, f'$R^2 = {r2:.2f}$', fontsize=13)
plt.text(10, 10, f'RMSE = {rmse:.2f}', fontsize=13)
plt.text(10, 11, f'MAE = {mae:.2f}', fontsize=13)

# Show the plot
plt.show()

In [ ]:
us_ts=us_results.copy()

In [ ]:
us_ts.reset_index(inplace=True)

In [ ]:
us_ts['date']=pd.to_datetime(us_ts['date'])

In [ ]:
us_ts=us_ts.set_index('date')

In [ ]:
columns_to_plot = ['p_preds', 'GPP_DT_VUT_REF', 'GBM_preds', 'RF_preds']


In [ ]:
# US-Var time series
plt.figure(figsize=(14, 8))


plt.plot(us_ts.index, us_ts['p_preds'], label='P-model', color='green', 
         linestyle=':', linewidth=1.5,alpha=0.7)


plt.plot(us_ts.index, us_ts['GBM_preds'], label='LightGBM', color='red', 
         linestyle=':', linewidth=1.5,alpha=0.7)

plt.plot(us_ts.index, us_ts['RF_preds'], label='RF', color='blue', 
         linestyle=':', linewidth=1.5,alpha=0.7)



plt.plot(us_ts.index, us_ts['GPP_DT_VUT_REF'], label='FLUXNET GPP', 
         color='gray', linestyle='-', linewidth=3)


plt.xlabel('Date', fontsize=14)
plt.ylabel('GPP (gC m$^{-2}$ d$^{-1}$)', fontsize=14)

plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=6))  
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  

plt.xticks(rotation=0)

plt.grid(True)
plt.legend(loc='upper right', fontsize=12)

plt.tight_layout()

plt.show()